In [1]:
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET


In [2]:
people_base = pd.read_xml('dataset.xml')
print(people_base.shape)

people_base.columns[-1:-8:-1]
people_base.drop(people_base.columns[-1:-8:-1], axis=1, inplace=True)
people_base.sort_values('UserID', inplace=True)
people_base.head()

(741, 16)


,UserID,Age,Gender,Living,YearsOfMusicEducation,YearsOfInstrumentPlaying,MusicListeningPerDay,TakeMoodDrugs,UnderInfluenceOfDrugs
0,1,27,Male,Countryside,0,0,3,Yes,Yes
261,2,22,Female,Countryside,7,15,4,No,No
176,3,21,Male,City,7,10,3,No,No
136,4,25,Female,City,8,8,4,No,No
30,5,24,Male,Countryside,6,11,2,No,No


In [3]:
tree = ET.parse('dataset.xml')
root = tree.getroot()


def nested_extract(node):
    if (list(node)):
        final = [nested_extract(elem) for elem in node]
        return final
    
    return node.text


columns = []
for item in root:
    child1 = ET.Element(root)
    child1 = item
    for cols in item:
        columns.append(cols.tag)
    break
len(columns)

    
tree = nested_extract(root)
tree = pd.DataFrame(tree, columns=columns)

tree_cp = tree.copy()
tree_cp.head()

,UserID,Age,Gender,Living,YearsOfMusicEducation,YearsOfInstrumentPlaying,MusicListeningPerDay,TakeMoodDrugs,UnderInfluenceOfDrugs,PreferredGenres,CurrentMoodVA,CurrentMoodColor,CurrentMoodA,EmotionVA,EmotionHSV,Songs
0,1,27,Male,Countryside,0,0,3,Yes,Yes,"[Pop, Metal, Opera]","[0.8885, 0.8262]","[0.825, 0.5, 1]","[0.8914, 0.6071, 0.0514, 0.0314, 0.03, 0.0129,...","[[-0.6164, -0.1049], [0.8197, 0.9574], [0.2885...","[[0.55, 1, 1], [0, 0.5, 1], [0.641666666666667...","[[101, [['0.9246', '0.8918']], [['0.4393', '-0..."
1,13,20,Female,Countryside,8,14,4,No,No,"[Rock, Alternative, Pop]","[0.4754, -0.1639]","[0.366666666666667, 1, 1]","[0.7129, 0.6571, 0.2786, 0.2971, 0.0457, 0.024...","[[-0.6295, 0.2361], [0.8197, 0.859], [-0.5639,...","[[0, 0, 0.5], [0, 0.75, 1], [0.641666666666667...","[[101, [['-0.5705', '-0.5115'], ['0.6033', '0...."
2,86,50,Male,Countryside,0,0,3,No,No,"[Rock, Metal, R&B/Soul]","[0.6393, 0.7279]","[0.641666666666667, 1, 1]","[0.6757, 0.7657, 0.1486, 0.0357, 0.5014, 0.604...","[[-0.3607, 0.7738], [0.5902, -0.6492], [0.7869...","[[0, 0, 0.25], [0, 1, 1], [0, 0, 0], [0, 0, 0....","[[101, [['0.5967', '-0.4066'], ['-0.0656', '0...."
3,124,23,Female,City,5,0,2,No,No,"[Reggae, Rock, Latin]","[-0.0033, -0.1639]","[0.641666666666667, 0.5, 1]","[0.0543, 0.1843, 0.9429, 0.93, 0.0571, 0.1229,...","[[-0.9836, 0.918], [0.7213, 0.8], [-0.2689, 0....","[[0.0916666666666667, 0.5, 1], [0, 1, 1], [0.7...","[[101, [['-0.8525', '-0.8393'], ['0.977', '0.7..."
4,207,23,Male,City,0,0,4,No,No,"[Electronica, Dance/Disco, Latin]","[0.6525, 0.3803]","[0.641666666666667, 0.75, 1]","[0.4557, 0.8014, 0.1314, 0.4543, 0.2114, 0.068...","[[-0.7016, 0.3475], [0.282, 0.8328], [-0.4721,...","[[0, 0.5, 1], [0, 0, 0], [0, 0, 0.25], [0.9166...","[[101, [['-0.3803', '-0.282'], ['0.4656', '0.4..."


In [4]:
tree_cp.columns

Index(['UserID', 'Age', 'Gender', 'Living', 'YearsOfMusicEducation',
       'YearsOfInstrumentPlaying', 'MusicListeningPerDay', 'TakeMoodDrugs',
       'UnderInfluenceOfDrugs', 'PreferredGenres', 'CurrentMoodVA',
       'CurrentMoodColor', 'CurrentMoodA', 'EmotionVA', 'EmotionHSV', 'Songs'],
      dtype='object')

In [37]:
composite = [
    'PreferredGenres', 
    'CurrentMoodVA', 
    'CurrentMoodColor', 
    'CurrentMoodA', 
    'EmotionVA', 
    'EmotionHSV', 
    'Songs'
]


# tree_cp = 

genre_df = pd.DataFrame({'UserID': tree_cp['UserID'].astype('int64')})
genre_exploded = tree_cp['PreferredGenres'].explode()

genre_df = genre_df.join(pd.crosstab(genre_exploded.index, genre_exploded))
genre_df.sort_values('UserID', inplace=True)
genre_df

,UserID,Alternative,Blues,Classical,Country,Dance/Disco,Easy Listening,Electronica,Folk,Hip Hop/Rap,...,Latin,Metal,New Age,Opera,Pop,Punk,R&B/Soul,Reggae,Rock,Vocal
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,0,0,0,0,0
261,2,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
176,3,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
136,4,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
30,5,0,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,1381,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
62,1382,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
205,1383,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
175,1392,0,0,0,0,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0


In [34]:
pd.crosstab(genre_exploded.index, genre_exploded)

PreferredGenres,Alternative,Blues,Classical,Country,Dance/Disco,Easy Listening,Electronica,Folk,Hip Hop/Rap,Jazz,Latin,Metal,New Age,Opera,Pop,Punk,R&B/Soul,Reggae,Rock,Vocal
row_0,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0
4,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
737,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
738,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [36]:
song = np.asarray(tree_cp['Songs'][0], dtype='object')
pd.DataFrame(song)


,0,1,2,3
0,101,"[[0.9246, 0.8918]]","[[0.4393, -0.6361]]","[0, 1, 1]"
1,153,"[[0.2885, 0.7344]]","[[0.0262, 0.2754]]","[0, 1, 1]"
2,209,"[[-0.8656, 0.6689]]","[[-0.5705, -0.4918]]","[0.458333333333333, 1, 1]"
3,242,"[[-0.6295, 0.2426], [-0.4787, 0.6033]]","[[-0.0852, -0.7672], [-0.4984, 0.3279]]","[0.0916666666666667, 0.25, 1]"
4,504,"[[0.8787, 0.8066], [0.9377, 0.8656]]","[[0.3148, 0.3869]]","[0, 0, 0.25]"
5,513,"[[0.3803, 0.7344]]","[[-0.7082, 0.2557]]","[0, 0.5, 1]"
6,525,"[[0.741, 0.6754]]","[[0.282, 0.4]]","[0.55, 0.75, 1]"
7,551,"[[0.341, 0.8]]","[[0.0459, -0.8]]","[0.183333333333333, 0.5, 1]"
8,607,"[[-0.7738, 0.9836]]","[[-0.8787, -0.5049]]","[0.55, 1, 1]"
9,618,"[[-0.5508, 0.3738]]","[[0.2557, -0.7803], [0.282, -0.7344]]","[0.733333333333333, 0.75, 1]"
